# **Setup**

In [ ]:
# Mount Google Drive:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Downgrade rpy:
!pip install rpy2==3.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rpy2: filename=rpy2-3.5.1-cp310-cp310-linux_x86_64.whl size=314948 sha256=c68cd3586282a2fbff3a340524b1d3dcdd72bdd8d8936a49ce925f2318a52b2f
  Stored in directory: /root/.cache/pip/wheels/73/a6/ff/4e75dd1ce1cfa2b9a670cbccf6a1e41c553199e9b25f05d953
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.4.2
    Uninstalling rpy2-3.4.2:
      Successfully uninstalled rpy2-3.4.2


In [ ]:
# Activate R magic:
%load_ext rpy2.ipython

In [ ]:
%%R
# Install and load the dplyr package:
install.packages("dplyr")
library(dplyr)

(as ‘lib’ is unspecified)







	‘/tmp/Rtmp7wxobg/downloaded_packages’

Attaching package: ‘dplyr’



    filter, lag



    intersect, setdiff, setequal, union




In [ ]:
%%R
# Install and load pander package:
install.packages("pander")
library(pander)

(as ‘lib’ is unspecified)







	‘/tmp/Rtmp7wxobg/downloaded_packages’



In [ ]:
%%R
# Install and load biomaRt:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("biomaRt")
library(biomaRt)

(as ‘lib’ is unspecified)







	‘/tmp/Rtmp7wxobg/downloaded_packages’

'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com





































































































	‘/tmp/Rtmp7wxobg/downloaded_packages’



# **Mt Robin**

## **All Genes**

In [ ]:
%%R
# Read the final table from Mt_Robin:
all_causal_genes <- read.table("/content/drive/MyDrive/Colab/Long_COVID/MtRobin/fs1_all_genes_sig_non_sig.txt", header=TRUE, sep="\t")
print(dim(all_causal_genes))
print(head(all_causal_genes))

[1] 1670   10
          gene_id gene_name      gene_type      beta_y Obs SNPs
1 ENSG00000002016     RAD52 protein_coding   0.8437588  23    8
2 ENSG00000005156      LIG3 protein_coding  12.6260663  10    3
3 ENSG00000006282   SPATA20 protein_coding -27.0963502  46    2
4 ENSG00000006744     ELAC2 protein_coding   7.5924973  18    4
5 ENSG00000007171      NOS2 protein_coding -20.1542226  31    5
6 ENSG00000007341      ST7L protein_coding 298.0706258  12    2
                                                                               SNP_IDs
1 rs11064536,rs11571383,rs117766797,rs147560086,rs2107614,rs4980935,rs718389,rs7955986
2                                                     rs12945428,rs201016783,rs3135958
3                                                                  rs2306001,rs9890200
4                                             rs1044564,rs11655929,rs733890,rs78840287
5                                    rs2314813,rs3751972,rs624196,rs72846720,rs8078340
6               

In [ ]:
%%R
# Read the results with the p_value and FDR values:
pvalue <- read.table("/content/drive/MyDrive/Colab/Long_COVID/MtRobin/MR_pvalue_FDR_results_fs1.txt", header=TRUE, sep="\t")
print(dim(pvalue))
print(head(pvalue))

[1] 1670    4
          gene_id pvalue nsamp_used       fdr
1 ENSG00000002016  0.884      10000 0.9779379
2 ENSG00000005156  0.143      10000 0.5553721
3 ENSG00000006282  0.126      10000 0.5367857
4 ENSG00000006744  0.140      10000 0.5514520
5 ENSG00000007171  0.467      10000 0.8385914
6 ENSG00000007341  0.286      10000 0.7280793


In [ ]:
%%R
# Merge both tables:
all_causal_genes_merged <- merge(all_causal_genes, pvalue, by = "gene_id", all = TRUE)
print(dim(all_causal_genes_merged))
print(head(all_causal_genes_merged))

[1] 1670   13
          gene_id gene_name      gene_type      beta_y Obs SNPs
1 ENSG00000002016     RAD52 protein_coding   0.8437588  23    8
2 ENSG00000005156      LIG3 protein_coding  12.6260663  10    3
3 ENSG00000006282   SPATA20 protein_coding -27.0963502  46    2
4 ENSG00000006744     ELAC2 protein_coding   7.5924973  18    4
5 ENSG00000007171      NOS2 protein_coding -20.1542226  31    5
6 ENSG00000007341      ST7L protein_coding 298.0706258  12    2
                                                                               SNP_IDs
1 rs11064536,rs11571383,rs117766797,rs147560086,rs2107614,rs4980935,rs718389,rs7955986
2                                                     rs12945428,rs201016783,rs3135958
3                                                                  rs2306001,rs9890200
4                                             rs1044564,rs11655929,rs733890,rs78840287
5                                    rs2314813,rs3751972,rs624196,rs72846720,rs8078340
6               

In [ ]:
%%R

# Load dplyr:
library(dplyr)

# Create a dataframe with the gene_id, p_value and FDR to calculate the final score:
mt_robin <- all_causal_genes_merged[, c("gene_id", "gene_name", "gene_type", "beta_y", "pvalue", "fdr")]
colnames(mt_robin)[colnames(mt_robin) == "pvalue"] <- "p_value"

# To print the results with 4 digist for the p_value and fdr columns:
mt_robin$p_value <- format(mt_robin$p_value, digits = 4, scientific = FALSE)
mt_robin$fdr <- format(mt_robin$fdr, digits = 4, scientific = FALSE)

# Sort table based on the lowest p_value:
mt_robin <- mt_robin[order(mt_robin$p_value), ]

# Print results:
print(dim(mt_robin))
print(head(mt_robin))

[1] 1670    6
            gene_id gene_name      gene_type    beta_y p_value     fdr
16  ENSG00000026950    BTN3A1 protein_coding -4.817862   0.000 0.00000
37  ENSG00000065457     ADAT1 protein_coding -7.713004   0.000 0.00000
63  ENSG00000090661     CERS4 protein_coding -7.804575   0.000 0.00000
156 ENSG00000113734     BNIP1 protein_coding  8.994436   0.000 0.00000
265 ENSG00000132507     EIF5A protein_coding 21.013660   0.000 0.00000
322 ENSG00000139714     MORN3 protein_coding  4.893023   0.000 0.00000


In [ ]:
%%R

# Count significant and non-significant genes:
significant_genes <- sum(mt_robin$p_value <= 0.05, na.rm = TRUE)
non_significant_genes <- sum(mt_robin$p_value > 0.05, na.rm = TRUE)

# Print results:
cat("\nNumber of significant genes (p-value <= 0.05):", significant_genes, "\n")
cat("Number of non-significant genes (p-value > 0.05):", non_significant_genes, "\n")


Number of significant genes (p-value <= 0.05): 198 
Number of non-significant genes (p-value > 0.05): 1472 


In [ ]:
%%R

# Create MtRobin_Score as the absolute value of beta_y:
mt_robin <- mt_robin %>%
  mutate(MR_Score = abs(beta_y))

# Print results:
print(dim(mt_robin))
print(head(mt_robin))

[1] 1670    7
            gene_id gene_name      gene_type    beta_y p_value     fdr
16  ENSG00000026950    BTN3A1 protein_coding -4.817862   0.000 0.00000
37  ENSG00000065457     ADAT1 protein_coding -7.713004   0.000 0.00000
63  ENSG00000090661     CERS4 protein_coding -7.804575   0.000 0.00000
156 ENSG00000113734     BNIP1 protein_coding  8.994436   0.000 0.00000
265 ENSG00000132507     EIF5A protein_coding 21.013660   0.000 0.00000
322 ENSG00000139714     MORN3 protein_coding  4.893023   0.000 0.00000
     MR_Score
16   4.817862
37   7.713004
63   7.804575
156  8.994436
265 21.013660
322  4.893023


In [ ]:
%%R

# Count the number of NAs in the gene_type column:
num_na_gene_type <- sum(is.na(mt_robin$gene_type))

# Print the result:
cat("Number of NAs in the gene_type column:", num_na_gene_type, "\n")

# Filter the rows where gene_type is NA
na_gene_type_rows <- mt_robin %>% filter(is.na(gene_type))

# Print results:
print(head(na_gene_type_rows))

Number of NAs in the gene_type column: 608 
          gene_id      gene_name gene_type      beta_y p_value     fdr
1 ENSG00000180347        CCDC129      <NA>   -4.856468   0.000 0.00000
2 ENSG00000229413 RP11-274B21.12      <NA> -100.134521   0.000 0.00000
3 ENSG00000230715  RP11-274B21.4      <NA> -106.769177   0.000 0.00000
4 ENSG00000240291  RP11-499P20.2      <NA>   14.706343   0.000 0.00000
5 ENSG00000248508      SRP14-AS1      <NA>  368.778750   0.000 0.00000
6 ENSG00000250271   RP11-64D22.5      <NA>  -43.911325   0.000 0.00000
    MR_Score
1   4.856468
2 100.134521
3 106.769177
4  14.706343
5 368.778750
6  43.911325


In [ ]:
%%R

# First conditional: If gene_type is NA and gene_name matches the first pattern
mt_robin$gene_type <- ifelse(
  is.na(mt_robin$gene_type) & grepl("^R.*-.*\\.", mt_robin$gene_name),
  "pseudogene",
  mt_robin$gene_type
)

# Second conditional: If gene_type is NA and gene_name matches the second pattern
mt_robin$gene_type <- ifelse(
  is.na(mt_robin$gene_type) & grepl(".*-.*|.*\\.", mt_robin$gene_name),
  "novel_transcript",
  mt_robin$gene_type
)

# Apply the third conditional: For the rest NAs, add the protein_coding label
mt_robin$gene_type <- ifelse(
  is.na(mt_robin$gene_type),
  "protein_coding",
  mt_robin$gene_type
)

# Count the number of NAs in the gene_type column:
num_na_gene_type <- sum(is.na(mt_robin$gene_type))

# Print the result:
cat("Number of NAs in the gene_type column:", num_na_gene_type, "\n")

# Print results:
print(dim(mt_robin))
print(head(mt_robin))

Number of NAs in the gene_type column: 0 
[1] 1670    7
            gene_id gene_name      gene_type    beta_y p_value     fdr
16  ENSG00000026950    BTN3A1 protein_coding -4.817862   0.000 0.00000
37  ENSG00000065457     ADAT1 protein_coding -7.713004   0.000 0.00000
63  ENSG00000090661     CERS4 protein_coding -7.804575   0.000 0.00000
156 ENSG00000113734     BNIP1 protein_coding  8.994436   0.000 0.00000
265 ENSG00000132507     EIF5A protein_coding 21.013660   0.000 0.00000
322 ENSG00000139714     MORN3 protein_coding  4.893023   0.000 0.00000
     MR_Score
16   4.817862
37   7.713004
63   7.804575
156  8.994436
265 21.013660
322  4.893023


In [ ]:
%%R

# Add the Risk, Preventive, or Non_Significant column to the mt_robin table:
mt_robin <- mt_robin %>%
  mutate(Gene_Effect = case_when(
    p_value > 0.05 | fdr > 0.05 ~ "Non_Significant",
    beta_y > 0 ~ "Risk",
    TRUE ~ "Preventive"
  ))

# Print results:
print(dim(mt_robin))
print(head(mt_robin))

[1] 1670    8
            gene_id gene_name      gene_type    beta_y p_value     fdr
16  ENSG00000026950    BTN3A1 protein_coding -4.817862   0.000 0.00000
37  ENSG00000065457     ADAT1 protein_coding -7.713004   0.000 0.00000
63  ENSG00000090661     CERS4 protein_coding -7.804575   0.000 0.00000
156 ENSG00000113734     BNIP1 protein_coding  8.994436   0.000 0.00000
265 ENSG00000132507     EIF5A protein_coding 21.013660   0.000 0.00000
322 ENSG00000139714     MORN3 protein_coding  4.893023   0.000 0.00000
     MR_Score Gene_Effect
16   4.817862  Preventive
37   7.713004  Preventive
63   7.804575  Preventive
156  8.994436        Risk
265 21.013660        Risk
322  4.893023        Risk


In [ ]:
%%R

# Count the number of genes in each category before deleting those that are not protein-coding genes:
gene_effect_counts <- table(mt_robin$Gene_Effect)
print(gene_effect_counts)

# Calculate percentages:
gene_effect_percentages <- prop.table(gene_effect_counts) * 100
print(round(gene_effect_percentages, 2))


Non_Significant      Preventive            Risk 
           1634              16              20 

Non_Significant      Preventive            Risk 
          97.84            0.96            1.20 


In [ ]:
%%R

# Delete rows which are not protein coding genes:
mt_robin <- mt_robin %>%
  filter(gene_type == "protein_coding")

# Print results:
print(dim(mt_robin))
print(head(mt_robin))

[1] 808   8
          gene_id gene_name      gene_type    beta_y p_value     fdr  MR_Score
1 ENSG00000026950    BTN3A1 protein_coding -4.817862   0.000 0.00000  4.817862
2 ENSG00000065457     ADAT1 protein_coding -7.713004   0.000 0.00000  7.713004
3 ENSG00000090661     CERS4 protein_coding -7.804575   0.000 0.00000  7.804575
4 ENSG00000113734     BNIP1 protein_coding  8.994436   0.000 0.00000  8.994436
5 ENSG00000132507     EIF5A protein_coding 21.013660   0.000 0.00000 21.013660
6 ENSG00000139714     MORN3 protein_coding  4.893023   0.000 0.00000  4.893023
  Gene_Effect
1  Preventive
2  Preventive
3  Preventive
4        Risk
5        Risk
6        Risk


In [ ]:
%%R

# Count the number of genes in each category after deleting those that are not protein-coding genes:
gene_effect_counts <- table(mt_robin$Gene_Effect)
print(gene_effect_counts)

# Calculate percentages:
gene_effect_percentages <- prop.table(gene_effect_counts) * 100
print(round(gene_effect_percentages, 2))


Non_Significant      Preventive            Risk 
            792               7               9 

Non_Significant      Preventive            Risk 
          98.02            0.87            1.11 


In [ ]:
%%R

# Assign 0 to MR_Score if p_value or fdr is > 0.05 (not significant) and normalize:
mt_robin <- mt_robin %>%
  mutate(
    MR_Score = ifelse(p_value > 0.05 | fdr > 0.05, 0, MR_Score),
    MR_Score_norm = (MR_Score - min(MR_Score, na.rm = TRUE)) /
                    (max(MR_Score, na.rm = TRUE) - min(MR_Score, na.rm = TRUE))
  ) %>%
  arrange(desc(MR_Score))

# Print results:
print(dim(mt_robin))
print(head(mt_robin))

[1] 808   9
          gene_id gene_name      gene_type     beta_y p_value     fdr  MR_Score
1 ENSG00000171160     MORN4 protein_coding  34.218202   0.000 0.00000 34.218202
2 ENSG00000176386     CDC26 protein_coding -30.039865   0.000 0.00000 30.039865
3 ENSG00000132507     EIF5A protein_coding  21.013660   0.000 0.00000 21.013660
4 ENSG00000146530      VWDE protein_coding  15.508145   0.000 0.00000 15.508145
5 ENSG00000173540     GMPPB protein_coding  13.284320   0.000 0.00000 13.284320
6 ENSG00000184983    NDUFA6 protein_coding   9.789797   0.000 0.00000  9.789797
  Gene_Effect MR_Score_norm
1        Risk     1.0000000
2  Preventive     0.8778914
3        Risk     0.6141077
4        Risk     0.4532133
5        Risk     0.3882238
6        Risk     0.2860991


In [ ]:
%%R

# Print the min and max for MR_Score_norm:
mtrobin_min <- min(mt_robin$MR_Score_norm, na.rm = TRUE)
mtrobin_max <- max(mt_robin$MR_Score_norm, na.rm = TRUE)
cat("MR_Score_norm - Min:", mtrobin_min, "Max:", mtrobin_max, "\n")

MR_Score_norm - Min: 0 Max: 1 


In [ ]:
%%R

# Check for NAs in the entire table:
any_na <- any(is.na(mt_robin))

# Display the result:
if (any_na) {
  print("There are NA values in the mt_robin table.")
} else {
  print("There are no NA values in the mt_robin table.")
}

# Check which columns contain NAs:
na_columns <- colnames(mt_robin)[apply(mt_robin, 2, anyNA)]
if (length(na_columns) > 0) {
  print(paste("Columns with NAs:", paste(na_columns, collapse = ", ")))
} else {
  print("No columns have NA values.")
}

[1] "There are no NA values in the mt_robin table."
[1] "No columns have NA values."


In [ ]:
%%R

# Define the folder to save the file:
output_folder <- "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final"

# Define the file name and path:
output_file <- file.path(output_folder, "mt_robin_sig_non_sig.csv")

# Save the mt_robin_sig data frame as a CSV file in the specified folder
write.csv(mt_robin, output_file, row.names = FALSE)

# Print confirmation
cat("Selected genes saved to:", output_file, "\n")

Selected genes saved to: /content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/mt_robin_sig_non_sig.csv 


## **Significant Genes**

In [ ]:
%%R

# Define the folder and file name where the data is stored:
input_folder <- "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final"
input_file <- file.path(input_folder, "mt_robin_sig_non_sig.csv")

# Read the CSV file into a data frame:
causal_sig_genes <- read.csv(input_file)

# Print the first few rows to verify:
print(dim(causal_sig_genes))
print(head(causal_sig_genes))

[1] 808   9
          gene_id gene_name      gene_type     beta_y p_value fdr  MR_Score
1 ENSG00000171160     MORN4 protein_coding  34.218202       0   0 34.218202
2 ENSG00000176386     CDC26 protein_coding -30.039865       0   0 30.039865
3 ENSG00000132507     EIF5A protein_coding  21.013660       0   0 21.013660
4 ENSG00000146530      VWDE protein_coding  15.508145       0   0 15.508145
5 ENSG00000173540     GMPPB protein_coding  13.284320       0   0 13.284320
6 ENSG00000184983    NDUFA6 protein_coding   9.789797       0   0  9.789797
  Gene_Effect MR_Score_norm
1        Risk     1.0000000
2  Preventive     0.8778914
3        Risk     0.6141077
4        Risk     0.4532133
5        Risk     0.3882238
6        Risk     0.2860991


In [ ]:
%%R

# Filter out rows where p_value and fdr > 0.05:
filtered_data <- causal_sig_genes %>%
  filter(p_value <= 0.05 & fdr <= 0.05)

# Order the table based on MR_Score from highest to lowest:
mt_robin_sig <- filtered_data %>%
  arrange(desc(MR_Score))

# Print results:
print(dim(mt_robin_sig))
print(head(mt_robin_sig))

[1] 16  9
          gene_id gene_name      gene_type     beta_y p_value fdr  MR_Score
1 ENSG00000171160     MORN4 protein_coding  34.218202       0   0 34.218202
2 ENSG00000176386     CDC26 protein_coding -30.039865       0   0 30.039865
3 ENSG00000132507     EIF5A protein_coding  21.013660       0   0 21.013660
4 ENSG00000146530      VWDE protein_coding  15.508145       0   0 15.508145
5 ENSG00000173540     GMPPB protein_coding  13.284320       0   0 13.284320
6 ENSG00000184983    NDUFA6 protein_coding   9.789797       0   0  9.789797
  Gene_Effect MR_Score_norm
1        Risk     1.0000000
2  Preventive     0.8778914
3        Risk     0.6141077
4        Risk     0.4532133
5        Risk     0.3882238
6        Risk     0.2860991


In [ ]:
%%R

# Define the folder to save the file:
output_folder <- "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final"

# Define the file name and path:
output_file <- file.path(output_folder, "mt_robin_sig.csv")

# Save the mt_robin_sig data frame as a CSV file in the specified folder
write.csv(mt_robin_sig, output_file, row.names = FALSE)

# Print confirmation
cat("Selected genes saved to:", output_file, "\n")

Selected genes saved to: /content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/mt_robin_sig.csv 


# **Control Theory**

## **All Indispensable Genes**

**Types of Nodes:**
- Neutral
- Dispensable
- Indispensable:
  - Type I:
    - Critical
    - Redundant
    - Ordinary
  - Type II
    - Critical
    - Redundant
    - Ordinary

**Indispensable nodes:**

**Type I and Type II Genes:** This refers to the classification of nodes in the network based on their role in controlling the system.
- Type-I and II node classification: 0 (critical), 1(redundant), 2 (ordinary)

**Type-I:**
- **Critical (0):** In its absence we have to control more driver nodes. For example, removing one node in the middle of a directed path will cause N_D increase.
- **Redundant (1):** In its absence we need to control less driver nodes. For example, removing one leaf node will decrease N_D by 1.
- **Ordinary (2):** In its absence we need to control the same number of driver nodes. For example, removing the central node will not change N_D at all.

**Type-II:**
- **Critical (0):** It belongs to all the driver node sets, i.e. we have to always control it. We can rigorously prove that a node is critical if and only if it has zero in-degree, i.e. kin=0.
- **Redundant (1):** It belongs to no driver node sets, so we never need to control them.
- **Ordinary (2):** It belongs to some but not all driver node sets.

In [ ]:
%%R
# Read the output table from Control_Theory:
# Note: From this dataset ALL genes are Indispensable (Type I or II)
nodetype_data <- read.table("/content/drive/MyDrive/Colab/Long_COVID/Control_Theory/GSE215865/PPI_PASC/procedure1/filtered_ppi_network_PASC.dat.nodetype", header=FALSE, sep=" ")
print(dim(nodetype_data))
print(head(nodetype_data))

[1] 6327    6
        V1  V2 V3 V4 V5 V6
1 C1orf103 117 64 53  0  1
2    MNAT1  24 16  8  0  1
3    MAST2  11  5  6  0  1
4   DYNLL1  60 35 25  0  1
5   RAB22A   3  1  2  2  2
6    APPL2   5  3  2  2  1


In [ ]:
%%R

# Rename columns (those names are in the original nodetype dataset):
colnames(nodetype_data) <- c("Name", "K", "Kin", "Kout", "TypeI", "TypeII")

# Print results:
print(dim(nodetype_data))
print(head(nodetype_data))

[1] 6327    6
      Name   K Kin Kout TypeI TypeII
1 C1orf103 117  64   53     0      1
2    MNAT1  24  16    8     0      1
3    MAST2  11   5    6     0      1
4   DYNLL1  60  35   25     0      1
5   RAB22A   3   1    2     2      2
6    APPL2   5   3    2     2      1


In [ ]:
%%R

# Change variable name from nodetype_data to control_theory
control_theory <- nodetype_data

# Change name column:
colnames(control_theory)[colnames(control_theory) == "Name"] <- "gene_name"

# Print results:
print(dim(control_theory))
print(head(control_theory))

[1] 6327    6
  gene_name   K Kin Kout TypeI TypeII
1  C1orf103 117  64   53     0      1
2     MNAT1  24  16    8     0      1
3     MAST2  11   5    6     0      1
4    DYNLL1  60  35   25     0      1
5    RAB22A   3   1    2     2      2
6     APPL2   5   3    2     2      1


In [ ]:
%%R
# Read the final table from Mt_Robin:
all_causal_genes <- read.table("/content/drive/MyDrive/Colab/Long_COVID/MtRobin/fs1_all_genes_sig_non_sig.txt", header=TRUE, sep="\t")
print(dim(all_causal_genes))
print(head(all_causal_genes))

[1] 1670   10
          gene_id gene_name      gene_type      beta_y Obs SNPs
1 ENSG00000002016     RAD52 protein_coding   0.8437588  23    8
2 ENSG00000005156      LIG3 protein_coding  12.6260663  10    3
3 ENSG00000006282   SPATA20 protein_coding -27.0963502  46    2
4 ENSG00000006744     ELAC2 protein_coding   7.5924973  18    4
5 ENSG00000007171      NOS2 protein_coding -20.1542226  31    5
6 ENSG00000007341      ST7L protein_coding 298.0706258  12    2
                                                                               SNP_IDs
1 rs11064536,rs11571383,rs117766797,rs147560086,rs2107614,rs4980935,rs718389,rs7955986
2                                                     rs12945428,rs201016783,rs3135958
3                                                                  rs2306001,rs9890200
4                                             rs1044564,rs11655929,rs733890,rs78840287
5                                    rs2314813,rs3751972,rs624196,rs72846720,rs8078340
6               

In [ ]:
%%R

# Merge the control_theory table with the all_causal_genes table based on the gene_name column:
control_theory <- merge(control_theory, all_causal_genes[, c("gene_name", "gene_id")],
                        by = "gene_name",
                        all.x = TRUE)

# Reorder the columns to place gene_id at the beginning
control_theory <- control_theory[, c("gene_id", setdiff(names(control_theory), "gene_id"))]

# Print results:
print(dim(control_theory))
print(head(control_theory))

[1] 6327    7
  gene_id gene_name  K Kin Kout TypeI TypeII
1    <NA>    101543  2   1    1     1      2
2    <NA>    127557 19  14    5     0      1
3    <NA>    150421  2   1    1     1      2
4    <NA>    151568  4   2    2     1      2
5    <NA>     16534  2   1    1     1      2
6    <NA>    220766  7   5    2     2      1


In [ ]:
%%R

# Count the number of rows where gene_id is NA:
num_missing_gene_id <- sum(is.na(control_theory$gene_id))

# Print the result:
cat("Number of genes without a gene_id:", num_missing_gene_id, "\n")

Number of genes without a gene_id: 6155 


In [ ]:
%%R

# Load the biomaRt package if necessary:
#library(biomaRt)

# Connect to the Ensembl biomart for the specified release (version 109):
ensembl <- useEnsembl(biomart = "genes", dataset = "hsapiens_gene_ensembl", version = 109)

# Check for missing gene_id and fetch only those gene_names
missing_genes <- control_theory$gene_name[is.na(control_theory$gene_id)]

# Only perform the lookup if there are missing gene_ids
if (length(missing_genes) > 0) {
  # Retrieve Ensembl gene IDs based on gene names only for missing gene_id
  gene_mapping <- getBM(
    attributes = c("ensembl_gene_id", "hgnc_symbol"),
    filters = "hgnc_symbol",
    values = unique(missing_genes),  # Ensure unique gene names
    mart = ensembl
  )

  # Remove any duplicates in gene_mapping to avoid creating multiple rows per gene_name
  gene_mapping <- gene_mapping[!duplicated(gene_mapping$hgnc_symbol), ]

  # Merge the gene mapping with the control_theory table:
  control_theory <- merge(control_theory, gene_mapping,
                          by.x = "gene_name", by.y = "hgnc_symbol", all.x = TRUE)

  # Fill in the missing gene_id values
  control_theory$gene_id <- ifelse(is.na(control_theory$gene_id),
                                   control_theory$ensembl_gene_id,
                                   control_theory$gene_id)

  # Drop the ensembl_gene_id column
  control_theory <- control_theory %>%
    dplyr::select(gene_id, gene_name, K, Kin, Kout, TypeI, TypeII)
}

# Print results:
print(dim(control_theory))
print(head(control_theory))

[1] 6327    7
  gene_id gene_name  K Kin Kout TypeI TypeII
1    <NA>    101543  2   1    1     1      2
2    <NA>    127557 19  14    5     0      1
3    <NA>    150421  2   1    1     1      2
4    <NA>    151568  4   2    2     1      2
5    <NA>     16534  2   1    1     1      2
6    <NA>    220766  7   5    2     2      1


In [ ]:
%%R

# Count the number of rows where gene_id is NA:
num_na_gene_id <- sum(is.na(control_theory$gene_id))

# Print the result:
cat("Number of genes without a gene_id:", num_na_gene_id, "\n")

Number of genes without a gene_id: 451 


In [ ]:
%%R

# Calculate the Weight and CT_Score:
control_theory <- control_theory %>%
  mutate(
    Weight = case_when(
      TypeI == 0 & TypeII != 0 ~ 2,
      TypeI != 0 & TypeII == 0 ~ 1,
      TypeI != 0 | TypeII != 0 ~ 0,
      TRUE ~ NA_real_  # This handles any unexpected cases
    ),
    CT_Score = K * Weight
  )

# Sorting the dataframe by CT_Score from highest to lowest:
control_theory <- control_theory %>%
  arrange(desc(CT_Score))

# Print results:
print(dim(control_theory))
print(head(control_theory, 10))

[1] 6327    9
           gene_id gene_name   K Kin Kout TypeI TypeII Weight CT_Score
1  ENSG00000141510      TP53 299 196  103     0      1      2      598
2  ENSG00000005339    CREBBP 273 153  120     0      1      2      546
3  ENSG00000100393     EP300 270 162  108     0      1      2      540
4  ENSG00000170027     YWHAG 252 180   72     0      1      2      504
5  ENSG00000166949     SMAD3 225 143   82     0      1      2      450
6  ENSG00000177885      GRB2 210  96  114     0      1      2      420
7  ENSG00000197122       SRC 195  92  103     0      1      2      390
8  ENSG00000169083        AR 179 112   67     0      1      2      358
9  ENSG00000091831      ESR1 174  68  106     0      1      2      348
10 ENSG00000139687       RB1 169 106   63     0      1      2      338


In [ ]:
%%R

# Add a new column 'gene_type' with the value 'protein_coding' for all rows:
control_theory <- control_theory %>%
  mutate(gene_type = "protein_coding")

# Print results:
print(dim(control_theory))
print(head(control_theory))

[1] 6327   10
          gene_id gene_name   K Kin Kout TypeI TypeII Weight CT_Score
1 ENSG00000141510      TP53 299 196  103     0      1      2      598
2 ENSG00000005339    CREBBP 273 153  120     0      1      2      546
3 ENSG00000100393     EP300 270 162  108     0      1      2      540
4 ENSG00000170027     YWHAG 252 180   72     0      1      2      504
5 ENSG00000166949     SMAD3 225 143   82     0      1      2      450
6 ENSG00000177885      GRB2 210  96  114     0      1      2      420
       gene_type
1 protein_coding
2 protein_coding
3 protein_coding
4 protein_coding
5 protein_coding
6 protein_coding


In [ ]:
%%R

# Normalize the values in the CT_Score column:
control_theory <- control_theory %>%
  mutate(
    CT_Score_norm = (CT_Score - min(CT_Score, na.rm = TRUE)) /
                    (max(CT_Score, na.rm = TRUE) - min(CT_Score, na.rm = TRUE))
  )

# Print results:
print(dim(control_theory))
print(head(control_theory))

[1] 6327   11
          gene_id gene_name   K Kin Kout TypeI TypeII Weight CT_Score
1 ENSG00000141510      TP53 299 196  103     0      1      2      598
2 ENSG00000005339    CREBBP 273 153  120     0      1      2      546
3 ENSG00000100393     EP300 270 162  108     0      1      2      540
4 ENSG00000170027     YWHAG 252 180   72     0      1      2      504
5 ENSG00000166949     SMAD3 225 143   82     0      1      2      450
6 ENSG00000177885      GRB2 210  96  114     0      1      2      420
       gene_type CT_Score_norm
1 protein_coding     1.0000000
2 protein_coding     0.9130435
3 protein_coding     0.9030100
4 protein_coding     0.8428094
5 protein_coding     0.7525084
6 protein_coding     0.7023411


In [ ]:
%%R

# Calculate the min and max for CT_Score_norm
min_CT_Score_norm <- min(control_theory$CT_Score_norm, na.rm = TRUE)
max_CT_Score_norm <- max(control_theory$CT_Score_norm, na.rm = TRUE)

# Print the results
cat("CT_Score_norm - Min:", min_CT_Score_norm, "Max:", max_CT_Score_norm, "\n")

CT_Score_norm - Min: 0 Max: 1 


In [ ]:
%%R

# Check for NAs in the entire table:
any_na <- any(is.na(control_theory))

# Display the result:
if (any_na) {
  print("There are NA values in the control_theory table.")
} else {
  print("There are no NA values in the control_theory table.")
}

# Check which columns contain NAs:
na_columns <- colnames(control_theory)[apply(control_theory, 2, anyNA)]
if (length(na_columns) > 0) {
  print(paste("Columns with NAs:", paste(na_columns, collapse = ", ")))
} else {
  print("No columns have NA values.")
}

[1] "There are NA values in the control_theory table."
[1] "Columns with NAs: gene_id"


In [ ]:
%%R

# Define the folder to save the file:
output_folder <- "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final"

# Define the file name and path:
output_file <- file.path(output_folder, "all_indispensable_genes.csv")

# Save the control_theory data frame as a CSV file in the specified folder
write.csv(control_theory, output_file, row.names = FALSE)

# Print confirmation
cat("Selected genes saved to:", output_file, "\n")

Selected genes saved to: /content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/all_indispensable_genes.csv 


## **Critical Genes**

In [ ]:
%%R

# Define the folder and file name where the data is stored
input_folder <- "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final"
input_file <- file.path(input_folder, "all_indispensable_genes.csv")

# Read the CSV file into a data frame
nodetype_data <- read.csv(input_file)

# Print the first few rows to verify
print(dim(nodetype_data))
print(head(nodetype_data))

[1] 6327   11
          gene_id gene_name   K Kin Kout TypeI TypeII Weight CT_Score
1 ENSG00000141510      TP53 299 196  103     0      1      2      598
2 ENSG00000005339    CREBBP 273 153  120     0      1      2      546
3 ENSG00000100393     EP300 270 162  108     0      1      2      540
4 ENSG00000170027     YWHAG 252 180   72     0      1      2      504
5 ENSG00000166949     SMAD3 225 143   82     0      1      2      450
6 ENSG00000177885      GRB2 210  96  114     0      1      2      420
       gene_type CT_Score_norm
1 protein_coding     1.0000000
2 protein_coding     0.9130435
3 protein_coding     0.9030100
4 protein_coding     0.8428094
5 protein_coding     0.7525084
6 protein_coding     0.7023411


In [ ]:
%%R

# All Critical Genes:
critical_nodes_TypeI_TypeII <- nodetype_data[
  nodetype_data$TypeI == 0 | nodetype_data$TypeII == 0,
]

# Extract nodes classified as critical for Type-I:
critical_nodes_TypeI <- nodetype_data[nodetype_data$TypeI == 0, ]

# Extract nodes classified as critical for Type-II:
critical_nodes_TypeII <- nodetype_data[nodetype_data$TypeII == 0, ]

# Print the datasets:
cat("Number of Type I and II critical nodes:", nrow(critical_nodes_TypeI_TypeII), "\n")
print(head(critical_nodes_TypeI_TypeII))
cat("Number of Type I critical nodes:", nrow(critical_nodes_TypeI), "\n")
print(head(critical_nodes_TypeI))
cat("Number of Type-II critical nodes:", nrow(critical_nodes_TypeII), "\n")
print(head(critical_nodes_TypeII))

Number of Type I and II critical nodes: 1721 
          gene_id gene_name   K Kin Kout TypeI TypeII Weight CT_Score
1 ENSG00000141510      TP53 299 196  103     0      1      2      598
2 ENSG00000005339    CREBBP 273 153  120     0      1      2      546
3 ENSG00000100393     EP300 270 162  108     0      1      2      540
4 ENSG00000170027     YWHAG 252 180   72     0      1      2      504
5 ENSG00000166949     SMAD3 225 143   82     0      1      2      450
6 ENSG00000177885      GRB2 210  96  114     0      1      2      420
       gene_type CT_Score_norm
1 protein_coding     1.0000000
2 protein_coding     0.9130435
3 protein_coding     0.9030100
4 protein_coding     0.8428094
5 protein_coding     0.7525084
6 protein_coding     0.7023411
Number of Type I critical nodes: 1334 
          gene_id gene_name   K Kin Kout TypeI TypeII Weight CT_Score
1 ENSG00000141510      TP53 299 196  103     0      1      2      598
2 ENSG00000005339    CREBBP 273 153  120     0      1      2      54

In [ ]:
%%R

# Count the number of Type I critical nodes:
typeI_critical <- sum(critical_nodes_TypeI_TypeII$TypeI == 0)
cat("Number of Type I critical nodes: ", typeI_critical, "\n")

# Count the number of Type II critical nodes:
typeII_critical <- sum(critical_nodes_TypeI_TypeII$TypeII == 0)
cat("Number of Type II critical nodes: ", typeII_critical, "\n")

# Count the number of Type I redundant nodes:
typeI_redundant <- sum(critical_nodes_TypeI_TypeII$TypeI == 1)
cat("Number of Type I redundant nodes: ", typeI_redundant, "\n")

# Count the number of Type II redundant nodes:
typeII_redundant <- sum(critical_nodes_TypeI_TypeII$TypeII == 1)
cat("Number of Type II redundant nodes: ", typeII_redundant, "\n")

# Count the number of Type I ordinary nodes:
typeI_ordinary <- sum(critical_nodes_TypeI_TypeII$TypeI == 2)
cat("Number of Type I ordinary nodes: ", typeI_ordinary, "\n")

# Count the number of Type II ordinary nodes:
typeII_ordinary <- sum(critical_nodes_TypeI_TypeII$TypeII == 2)
cat("Number of Type II ordinary nodes: ", typeII_ordinary, "\n")

Number of Type I critical nodes:  1334 
Number of Type II critical nodes:  387 
Number of Type I redundant nodes:  101 
Number of Type II redundant nodes:  1334 
Number of Type I ordinary nodes:  286 
Number of Type II ordinary nodes:  0 


In [ ]:
%%R

# Define the folder to save the file:
output_folder <- "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final"

# Define the file name and path:
output_file <- file.path(output_folder, "critical_nodes_TypeI_TypeII.csv")

# Save the critical_nodes_TypeI_TypeII data frame as a CSV file in the specified folder
write.csv(critical_nodes_TypeI_TypeII, output_file, row.names = FALSE)

# Print confirmation
cat("Selected genes saved to:", output_file, "\n")

Selected genes saved to: /content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/critical_nodes_TypeI_TypeII.csv 


# **Mt Robin & CT**

In [ ]:
%%R

# Define the folder and file name where the data is stored for the MR results:
input_folder <- "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final"
input_file <- file.path(input_folder, "mt_robin_sig.csv")

# Read the CSV file into a data frame:
mt_robin_all <- read.csv(input_file)

# Print the first few rows to verify:
print(dim(mt_robin_all))
print(mt_robin_all)

[1] 16  9
           gene_id gene_name      gene_type     beta_y p_value     fdr
1  ENSG00000171160     MORN4 protein_coding  34.218202   0.000 0.00000
2  ENSG00000176386     CDC26 protein_coding -30.039865   0.000 0.00000
3  ENSG00000132507     EIF5A protein_coding  21.013660   0.000 0.00000
4  ENSG00000146530      VWDE protein_coding  15.508145   0.000 0.00000
5  ENSG00000173540     GMPPB protein_coding  13.284320   0.000 0.00000
6  ENSG00000184983    NDUFA6 protein_coding   9.789797   0.000 0.00000
7  ENSG00000113734     BNIP1 protein_coding   8.994436   0.000 0.00000
8  ENSG00000090661     CERS4 protein_coding  -7.804575   0.000 0.00000
9  ENSG00000065457     ADAT1 protein_coding  -7.713004   0.000 0.00000
10 ENSG00000177025  C19orf18 protein_coding   7.706716   0.001 0.04639
11 ENSG00000183336     BOLA2 protein_coding  -6.115419   0.000 0.00000
12 ENSG00000158825       CDA protein_coding  -6.070831   0.000 0.00000
13 ENSG00000139714     MORN3 protein_coding   4.893023   0.000 0.00

In [ ]:
%%R

# Define the folder and file name where the data is stored for the CT results:
input_folder <- "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final"
input_file <- file.path(input_folder, "all_indispensable_genes.csv")

# Read the CSV file into a data frame:
all_indispensable_genes <- read.csv(input_file)

# Print the first few rows to verify:
print(dim(all_indispensable_genes))
print(head(all_indispensable_genes))

[1] 6327   11
          gene_id gene_name   K Kin Kout TypeI TypeII Weight CT_Score
1 ENSG00000141510      TP53 299 196  103     0      1      2      598
2 ENSG00000005339    CREBBP 273 153  120     0      1      2      546
3 ENSG00000100393     EP300 270 162  108     0      1      2      540
4 ENSG00000170027     YWHAG 252 180   72     0      1      2      504
5 ENSG00000166949     SMAD3 225 143   82     0      1      2      450
6 ENSG00000177885      GRB2 210  96  114     0      1      2      420
       gene_type CT_Score_norm
1 protein_coding     1.0000000
2 protein_coding     0.9130435
3 protein_coding     0.9030100
4 protein_coding     0.8428094
5 protein_coding     0.7525084
6 protein_coding     0.7023411


In [ ]:
%%R

# Merge the datasets based on both gene_id and gene_name with a full outer join:
mt_robin_CT <- merge(all_indispensable_genes, mt_robin_all, by = c("gene_id", "gene_name", "gene_type"), all = TRUE)

# Print results:
print(dim(mt_robin_CT))
print(head(mt_robin_CT))

[1] 6339   17
          gene_id gene_name      gene_type  K Kin Kout TypeI TypeII Weight
1 ENSG00000000003    TSPAN6 protein_coding  3   0    3     1      0      1
2 ENSG00000000938       FGR protein_coding 18   4   14     0      1      2
3 ENSG00000000971       CFH protein_coding  6   3    3     0      1      2
4 ENSG00000001167      NFYA protein_coding 30  16   14     0      1      2
5 ENSG00000001617    SEMA3F protein_coding  5   3    2     1      2      0
6 ENSG00000001626      CFTR protein_coding 17   5   12     2      2      0
  CT_Score CT_Score_norm beta_y p_value fdr MR_Score Gene_Effect MR_Score_norm
1        3   0.005016722     NA      NA  NA       NA        <NA>            NA
2       36   0.060200669     NA      NA  NA       NA        <NA>            NA
3       12   0.020066890     NA      NA  NA       NA        <NA>            NA
4       60   0.100334448     NA      NA  NA       NA        <NA>            NA
5        0   0.000000000     NA      NA  NA       NA        <NA>  

In [ ]:
%%R

# Count the number of repetitions for each gene_name
gene_name_repetitions <- mt_robin_CT %>%
  dplyr::group_by(gene_name) %>%
  dplyr::summarize(count = n()) %>%
  dplyr::filter(count > 1) %>%
  arrange(desc(count))

# Print the number of gene names with repetitions and a preview of the results
cat("Number of gene names with repetitions:", nrow(gene_name_repetitions), "\n")
print(head(gene_name_repetitions))

Number of gene names with repetitions: 0 
# A tibble: 0 × 2
# ℹ 2 variables: gene_name <chr>, count <int>


In [ ]:
%%R

# Check for duplicate gene_name entries:
duplicate_genes <- mt_robin_CT %>%
  group_by(gene_name) %>%
  filter(n() > 1)

# Print the number of duplicates found and view the duplicates:
num_duplicates <- nrow(duplicate_genes)
cat("Number of duplicate gene_name entries:", num_duplicates, "\n")

# If there are duplicates, display them:
if(num_duplicates > 0) {
  print(duplicate_genes)
}

# Calculate the number of unique genes in the mt_robin_CT:
num_unique_genes <- mt_robin_CT %>%
  distinct(gene_name) %>%
  nrow()

# Print the number of unique genes:
cat("Number of unique genes in the mt_robin_CT:", num_unique_genes, "\n")

Number of duplicate gene_name entries: 0 
Number of unique genes in the mt_robin_CT: 6339 


In [ ]:
%%R

# Change NA to 0 in MR_Score and CT_Score:
mt_robin_CT <- mt_robin_CT %>%
  mutate(
    MR_Score = ifelse(is.na(MR_Score), 0, MR_Score),
    CT_Score = ifelse(is.na(CT_Score), 0, CT_Score)
  )

# Print results:
print(dim(mt_robin_CT))
print(head(mt_robin_CT))

[1] 6339   17
          gene_id gene_name      gene_type  K Kin Kout TypeI TypeII Weight
1 ENSG00000000003    TSPAN6 protein_coding  3   0    3     1      0      1
2 ENSG00000000938       FGR protein_coding 18   4   14     0      1      2
3 ENSG00000000971       CFH protein_coding  6   3    3     0      1      2
4 ENSG00000001167      NFYA protein_coding 30  16   14     0      1      2
5 ENSG00000001617    SEMA3F protein_coding  5   3    2     1      2      0
6 ENSG00000001626      CFTR protein_coding 17   5   12     2      2      0
  CT_Score CT_Score_norm beta_y p_value fdr MR_Score Gene_Effect MR_Score_norm
1        3   0.005016722     NA      NA  NA        0        <NA>            NA
2       36   0.060200669     NA      NA  NA        0        <NA>            NA
3       12   0.020066890     NA      NA  NA        0        <NA>            NA
4       60   0.100334448     NA      NA  NA        0        <NA>            NA
5        0   0.000000000     NA      NA  NA        0        <NA>  

In [ ]:
%%R

# Check for NA values in each column:
na_counts <- colSums(is.na(mt_robin_CT))

# Print the number of NA values for each column:
print(na_counts)

      gene_id     gene_name     gene_type             K           Kin 
          451             0             0            12            12 
         Kout         TypeI        TypeII        Weight      CT_Score 
           12            12            12            12             0 
CT_Score_norm        beta_y       p_value           fdr      MR_Score 
           12          6323          6323          6323             0 
  Gene_Effect MR_Score_norm 
         6323          6323 


In [ ]:
%%R

# Define the folder to save the file:
output_folder <- "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final"

# Define the file name and path:
output_file <- file.path(output_folder, "mt_robin_CT.csv")

# Save the mt_robin_CT data frame as a CSV file in the specified folder:
write.csv(mt_robin_CT, output_file, row.names = FALSE)

# Print confirmation:
cat("Selected genes saved to:", output_file, "\n")

Selected genes saved to: /content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/mt_robin_CT.csv 


# **Final Display for User**

In [73]:
%%R

# Load necessary libraries:
library(dplyr)
library(knitr)
library(pander)

# Function to calculate Final_Score, summarize results, and output to a file:
generate_report <- function(file_path, alpha, num_genes_to_print = Inf, output_file) {

  # Step 1: Read the input data from the specified CSV file
  data <- read.csv(file_path)

  # Step 2: Check if MR_Score_norm and CT_Score_norm columns exist, if not create them
  if (!"MR_Score_norm" %in% colnames(data)) {
    data$MR_Score_norm <- NA
  }
  if (!"CT_Score_norm" %in% colnames(data)) {
    data$CT_Score_norm <- NA
  }

  # Step 3: Replace NA values in MR_Score_norm and CT_Score_norm with 0
  data <- data %>%
    mutate(
      MR_Score_norm = ifelse(is.na(MR_Score_norm), 0, MR_Score_norm),
      CT_Score_norm = ifelse(is.na(CT_Score_norm), 0, CT_Score_norm)
    )

  # Step 4: Calculate Final_Score based on the alpha parameter and select genes
  data <- data %>%
    mutate(
      Final_Score = case_when(
        !is.na(MR_Score_norm) & !is.na(CT_Score_norm) ~
          alpha * MR_Score_norm + (1 - alpha) * CT_Score_norm,
        !is.na(MR_Score_norm) ~ MR_Score_norm,
        !is.na(CT_Score_norm) ~ CT_Score_norm
      )
    )

  # Step 5: Select the top genes based on user input
  selected_genes <- data %>%
    arrange(desc(Final_Score)) %>%
    head(num_genes_to_print)

  # Step 6: Add a Rank column to indicate the order
  selected_genes <- selected_genes %>%
    mutate(Rank = row_number())

  # Step 7: Conditionally add the Gene_Effect column if MR_Score_norm != 0, else set to "No_Significant"
  selected_genes <- selected_genes %>%
    mutate(Gene_Effect = ifelse(MR_Score_norm != 0, Gene_Effect, "No_Significant"))

  # Step 8: Conditionally add the Critical_Gene column if CT_Score_norm != 0, else set to "Not_Critical"
  selected_genes <- selected_genes %>%
    mutate(Critical_Gene = case_when(
      TypeI == 0 ~ "TypeI",
      TypeII == 0 ~ "TypeII",
      TRUE ~ "Not_Critical"
    ))

  # Step 9: Calculate summary statistics for the selected genes
  total_genes <- nrow(data)
  num_risk_genes <- sum(selected_genes$Gene_Effect == "Risk", na.rm = TRUE)
  num_preventive_genes <- sum(selected_genes$Gene_Effect == "Preventive", na.rm = TRUE)
  num_typeI_critical <- sum(selected_genes$Critical_Gene == "TypeI", na.rm = TRUE)
  num_typeII_critical <- sum(selected_genes$Critical_Gene == "TypeII", na.rm = TRUE)
  total_selected_genes <- nrow(selected_genes)

  # Step 10: Print summary statistics for the selected genes
  cat("Summary of Selected Genes:\n")
  cat("Total Number of Protein-coding genes in the dataset:", total_genes, "\n")
  cat("Total Number of Selected Protein-coding genes:", total_selected_genes, "\n")
  cat("Number of Selected Risk genes:", num_risk_genes, "\n")
  cat("Number of Selected Preventive genes:", num_preventive_genes, "\n")
  cat("Number of Selected TypeI Critical genes:", num_typeI_critical, "\n")
  cat("Number of Selected TypeII Critical genes:", num_typeII_critical, "\n")

  # Step 11: Select the final columns based on conditions
  final_columns <- c("Rank", "gene_name", "MR_Score_norm", "CT_Score_norm", "Final_Score", "Gene_Effect", "Critical_Gene")

  # Step 12: Save the final table as a CSV file
  selected_genes %>%
    dplyr::select(all_of(final_columns)) %>%
    write.csv(output_file, row.names = FALSE)

  cat("Results saved to:", output_file, "\n")

  # Step 13: Display the first few rows of the final table
  selected_genes %>%
    dplyr::select(all_of(final_columns)) %>%
    kable()
}

## **Alpha = 1.00**

In [74]:
%%R
# Example 1: alpha = 1.00
generate_report(file_path = "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/mt_robin_CT.csv",
                alpha = 1.00,
                num_genes_to_print = 16,
                output_file = "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/alpha_1.00.csv")

Summary of Selected Genes:
Total Number of Protein-coding genes in the dataset: 6339 
Total Number of Selected Protein-coding genes: 16 
Number of Selected Risk genes: 9 
Number of Selected Preventive genes: 7 
Number of Selected TypeI Critical genes: 0 
Number of Selected TypeII Critical genes: 0 
Results saved to: /content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/alpha_1.00.csv 


| Rank|gene_name | MR_Score_norm| CT_Score_norm| Final_Score|Gene_Effect |Critical_Gene |
|----:|:---------|-------------:|-------------:|-----------:|:-----------|:-------------|
|    1|MORN4     |     1.0000000|             0|   1.0000000|Risk        |Not_Critical  |
|    2|CDC26     |     0.8778914|             0|   0.8778914|Preventive  |Not_Critical  |
|    3|EIF5A     |     0.6141077|             0|   0.6141077|Risk        |Not_Critical  |
|    4|VWDE      |     0.4532133|             0|   0.4532133|Risk        |Not_Critical  |
|    5|GMPPB     |     0.3882238|             0|   0.3882

## **Alpha = 0.75**

In [75]:
%%R
# Example 2: alpha = 0.75
generate_report(file_path = "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/mt_robin_CT.csv",
                alpha = 0.75,
                num_genes_to_print = 16,
                output_file = "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/alpha_0.75.csv")

Summary of Selected Genes:
Total Number of Protein-coding genes in the dataset: 6339 
Total Number of Selected Protein-coding genes: 16 
Number of Selected Risk genes: 7 
Number of Selected Preventive genes: 3 
Number of Selected TypeI Critical genes: 6 
Number of Selected TypeII Critical genes: 0 
Results saved to: /content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/alpha_0.75.csv 


| Rank|gene_name | MR_Score_norm| CT_Score_norm| Final_Score|Gene_Effect    |Critical_Gene |
|----:|:---------|-------------:|-------------:|-----------:|:--------------|:-------------|
|    1|MORN4     |     1.0000000|     0.0000000|   0.7500000|Risk           |Not_Critical  |
|    2|CDC26     |     0.8778914|     0.0000000|   0.6584185|Preventive     |Not_Critical  |
|    3|EIF5A     |     0.6141077|     0.0000000|   0.4605808|Risk           |Not_Critical  |
|    4|VWDE      |     0.4532133|     0.0000000|   0.3399100|Risk           |Not_Critical  |
|    5|GMPPB     |     0.3882238|     0

## **Alpha = 0.50**

In [76]:
%%R
# Example 3: alpha = 0.50
generate_report(file_path = "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/mt_robin_CT.csv",
                alpha = 0.50,
                num_genes_to_print = 16,
                output_file = "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/alpha_0.50.csv")

Summary of Selected Genes:
Total Number of Protein-coding genes in the dataset: 6339 
Total Number of Selected Protein-coding genes: 16 
Number of Selected Risk genes: 2 
Number of Selected Preventive genes: 1 
Number of Selected TypeI Critical genes: 13 
Number of Selected TypeII Critical genes: 0 
Results saved to: /content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/alpha_0.50.csv 


| Rank|gene_name | MR_Score_norm| CT_Score_norm| Final_Score|Gene_Effect    |Critical_Gene |
|----:|:---------|-------------:|-------------:|-----------:|:--------------|:-------------|
|    1|TP53      |     0.0000000|     1.0000000|   0.5000000|No_Significant |TypeI         |
|    2|MORN4     |     1.0000000|     0.0000000|   0.5000000|Risk           |Not_Critical  |
|    3|CREBBP    |     0.0000000|     0.9130435|   0.4565217|No_Significant |TypeI         |
|    4|EP300     |     0.0000000|     0.9030100|   0.4515050|No_Significant |TypeI         |
|    5|CDC26     |     0.8778914|     

## **Alpha = 0.25**

In [77]:
%%R
# Example 4: alpha = 0.25
generate_report(file_path = "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/mt_robin_CT.csv",
                alpha = 0.25,
                num_genes_to_print = 16,
                output_file = "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/alpha_0.25.csv")

Summary of Selected Genes:
Total Number of Protein-coding genes in the dataset: 6339 
Total Number of Selected Protein-coding genes: 16 
Number of Selected Risk genes: 0 
Number of Selected Preventive genes: 0 
Number of Selected TypeI Critical genes: 16 
Number of Selected TypeII Critical genes: 0 
Results saved to: /content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/alpha_0.25.csv 


| Rank|gene_name | MR_Score_norm| CT_Score_norm| Final_Score|Gene_Effect    |Critical_Gene |
|----:|:---------|-------------:|-------------:|-----------:|:--------------|:-------------|
|    1|TP53      |             0|     1.0000000|   0.7500000|No_Significant |TypeI         |
|    2|CREBBP    |             0|     0.9130435|   0.6847826|No_Significant |TypeI         |
|    3|EP300     |             0|     0.9030100|   0.6772575|No_Significant |TypeI         |
|    4|YWHAG     |             0|     0.8428094|   0.6321070|No_Significant |TypeI         |
|    5|SMAD3     |             0|     

## **Alpha = 0.00**

In [78]:
%%R
# Example 5: alpha = 0.00
generate_report(file_path = "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/mt_robin_CT.csv",
                alpha = 0.00,
                num_genes_to_print = 16,
                output_file = "/content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/alpha_0.00.csv")

Summary of Selected Genes:
Total Number of Protein-coding genes in the dataset: 6339 
Total Number of Selected Protein-coding genes: 16 
Number of Selected Risk genes: 0 
Number of Selected Preventive genes: 0 
Number of Selected TypeI Critical genes: 16 
Number of Selected TypeII Critical genes: 0 
Results saved to: /content/drive/MyDrive/Colab/Long_COVID/final_results/GitHub/Final/alpha_0.00.csv 


| Rank|gene_name | MR_Score_norm| CT_Score_norm| Final_Score|Gene_Effect    |Critical_Gene |
|----:|:---------|-------------:|-------------:|-----------:|:--------------|:-------------|
|    1|TP53      |             0|     1.0000000|   1.0000000|No_Significant |TypeI         |
|    2|CREBBP    |             0|     0.9130435|   0.9130435|No_Significant |TypeI         |
|    3|EP300     |             0|     0.9030100|   0.9030100|No_Significant |TypeI         |
|    4|YWHAG     |             0|     0.8428094|   0.8428094|No_Significant |TypeI         |
|    5|SMAD3     |             0|     